# FMA: A Dataset For Music Analysis

Michaël Defferrard, Kirell Benzi, Pierre Vandergheynst, Xavier Bresson, EPFL LTS2.

## Baselines

* This notebook evalutates standard classifiers from scikit-learn on the provided features.
* Moreover, it evaluates Deep Learning models on both audio and spectrograms.

In [37]:
import time
import os

import IPython.display as ipd
from tqdm import tqdm_notebook
import numpy as np
import pandas as pd
import keras
import tensorflow
import librosa
from keras.layers import Activation, Dense, Conv1D, Conv2D, MaxPooling1D, Flatten, Reshape

from sklearn.utils import shuffle
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder, LabelBinarizer, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC, LinearSVC
#from sklearn.gaussian_process import GaussianProcessClassifier
#from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.multiclass import OneVsRestClassifier

import utils

In [38]:
AUDIO_DIR = os.environ.get('AUDIO_DIR')

tracks = utils.load('../fma_metadata/tracks.csv')
features = utils.load('../fma_metadata/features.csv')
echonest = utils.load('../fma_metadata/echonest.csv')

# tracks = tracks['set', 'subset'] <= 'small'

np.testing.assert_array_equal(features.index, tracks.index)
assert echonest.index.isin(tracks.index).all()

tracks.shape, features.shape, echonest.shape

((106574, 52), (106574, 518), (13129, 249))

## Subset

In [39]:
subset = tracks.index[tracks['set', 'subset'] <= 'small']

assert subset.isin(tracks.index).all()
assert subset.isin(features.index).all()

features_all = features.join(echonest, how='inner').sort_index(axis=1)
print('Not enough Echonest features: {}'.format(features_all.shape))

tracks = tracks.loc[subset]
features_all = features.loc[subset]
tracks = tracks.drop([133297, 99134, 108925])

tracks.shape, features_all.shape

Not enough Echonest features: (13129, 767)


((7997, 52), (8000, 518))

In [40]:
train = tracks.index[tracks['set', 'split'] == 'training']
val = tracks.index[tracks['set', 'split'] == 'validation']
test = tracks.index[tracks['set', 'split'] == 'test']

print('{} training examples, {} validation examples, {} testing examples'.format(*map(len, [train, val, test])))

genres = list(LabelEncoder().fit(tracks['track', 'genre_top']).classes_)
#genres = list(tracks['track', 'genre_top'].unique())
print('Top genres ({}): {}'.format(len(genres), genres))
genres = list(MultiLabelBinarizer().fit(tracks['track', 'genres_all']).classes_)
print('All genres ({}): {}'.format(len(genres), genres))

6397 training examples, 800 validation examples, 800 testing examples
Top genres (8): ['Electronic', 'Experimental', 'Folk', 'Hip-Hop', 'Instrumental', 'International', 'Pop', 'Rock']
All genres (114): [1, 2, 6, 10, 12, 15, 16, 17, 18, 21, 22, 25, 26, 27, 30, 31, 32, 33, 36, 38, 41, 42, 45, 46, 47, 49, 53, 58, 64, 66, 70, 71, 76, 77, 79, 81, 83, 85, 86, 88, 89, 90, 92, 94, 98, 100, 101, 102, 103, 107, 109, 111, 113, 117, 118, 125, 130, 167, 171, 172, 174, 177, 180, 181, 182, 183, 184, 185, 186, 214, 224, 232, 236, 240, 247, 250, 267, 286, 296, 297, 314, 337, 359, 360, 361, 362, 400, 401, 404, 439, 440, 456, 468, 491, 495, 502, 504, 514, 524, 538, 539, 542, 580, 602, 619, 695, 741, 763, 808, 811, 1032, 1060, 1193, 1235]


## 2 Deep learning on raw audio

Other architectures:
* [Learning Features of Music from Scratch (MusicNet)](https://arxiv.org/abs/1611.09827), John Thickstun, Zaid Harchaoui, Sham Kakade.

Load audio samples in parallel using `multiprocessing` so as to maximize CPU usage when decoding MP3s and making some optional pre-processing. There are multiple ways to load a waveform from a compressed MP3:
* librosa uses audioread in the backend which can use many native libraries, e.g. ffmpeg
    * resampling is very slow --> use `kaiser_fast`
    * does not work with multi-processing, for keras `fit_generator()`
* pydub is a high-level interface for audio modification, uses ffmpeg to load
    * store a temporary `.wav`
* directly pipe ffmpeg output
    * fastest method
* [pyAV](https://github.com/mikeboers/PyAV) may be a fastest alternative by linking to ffmpeg libraries

In [41]:
# Just be sure that everything is fine. Multiprocessing is tricky to debug.
AUDIO_DIR = os.environ.get('AUDIO_DIR')
AUDIO_DIR='/home/hany/MVI/test/cropped_5s'
print(AUDIO_DIR)

# utils.LibrosaLoader().load(utils.get_audio_path(AUDIO_DIR, 2))
# SampleLoader = utils.build_sample_loader(AUDIO_DIR, labels_onehot, utils.FfmpegLoader())
# SampleLoader(train, batch_size=2).__next__()[0].shape

/home/hany/MVI/test/cropped_5s


In [42]:
# Keras parameters.
NB_WORKER = len(os.sched_getaffinity(0))  # number of usables CPUs
params = {'pickle_safe': True, 'nb_worker': NB_WORKER, 'max_q_size': 10}

### 2.1 Fully connected neural network

* Two layers with 10 hiddens is no better than random, ~11%.

Optimize data loading to be CPU / GPU bound, not IO bound. Larger batches means reduced training time, so increase batch time until memory exhaustion. Number of workers and queue size have no influence on speed.

In [43]:
# import time
# import os

# import IPython.display as ipd
# from tqdm import tqdm_notebook
# import numpy as np
# import pandas as pd
# import keras
# import tensorflow
# import librosa
# from keras.layers import Activation, Dense, Conv1D, Conv2D, MaxPooling1D, Flatten, Reshape

# from sklearn.utils import shuffle
# from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder, LabelBinarizer, StandardScaler
# from sklearn.linear_model import LogisticRegression
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.svm import SVC, LinearSVC
# #from sklearn.gaussian_process import GaussianProcessClassifier
# #from sklearn.gaussian_process.kernels import RBF
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
# from sklearn.neural_network import MLPClassifier
# from sklearn.naive_bayes import GaussianNB
# from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
# from sklearn.multiclass import OneVsRestClassifier

# import utils

In [44]:
# import warnings
# warnings.filterwarnings('ignore')

# AUDIO_DIR='/home/hany/MVI/test/fma_small'

# data = []
# x, sr = librosa.load('../fma_small/000/000002.mp3', duration=5)
# myshape = x.shape
# i = 0
# for root, dirs, files in os.walk(AUDIO_DIR):
#     for file in files:
#         i = i+1
#         print(root+'/'+file+'    '+str(i))
#         try:
#             x, sr = librosa.load(root+'/'+file, duration=5)
#         except:
#             continue
#         if x.shape != myshape:
#             print(f"myshape {myshape} != newshape {x.shape}")
#             continue
            
#         print(f'outputting {file}')
#         librosa.output.write_wav('../cropped_5s/'+file, x, sr)
        
# #         print('Duration: {:.2f}s, {} samples'.format(x.shape[-1] / sr, x.size))
# #     if i == 150:
# #         break
        
# #         start, end = 7, 17
# #         ipd.Audio(data=x[start*sr:end*sr], rate=sr)
# #         print(file)
# #         print(root)
# #         with open(os.path.join(root, file), "r") as f:


In [55]:
import dotenv
import pydot
import requests
import numpy as np
import pandas as pd
import ctypes
import shutil
import multiprocessing
import multiprocessing.sharedctypes as sharedctypes
import os.path
import ast

from pandas.api.types import CategoricalDtype

NB_AUDIO_SAMPLES = 7997
SAMPLING_RATE = 44100

In [60]:
def get_audio_path(audio_dir, track_id):
    tid_str = '{:06d}'.format(track_id)
    return os.path.join(audio_dir, tid_str + '.mp3')


In [61]:
class Loader:
    def load(self, filepath):
        raise NotImplemented()


class RawAudioLoader(Loader):
    def __init__(self, sampling_rate=SAMPLING_RATE):
        self.sampling_rate = sampling_rate
        self.shape = (NB_AUDIO_SAMPLES * sampling_rate // SAMPLING_RATE, )

    def load(self, filepath):
        return self._load(filepath)[:self.shape[0]]

import librosa
class LibrosaLoader(RawAudioLoader):
    def _load(self, filepath):
        sr = self.sampling_rate if self.sampling_rate != SAMPLING_RATE else None
        # kaiser_fast is 3x faster than kaiser_best
        #x, sr = librosa.load(filepath, sr=sr, res_type='kaiser_fast')
        x, sr = librosa.load(filepath, sr=sr)
        return x

import subprocess as sp
class FfmpegLoader(RawAudioLoader):
    def _load(self, filepath):
        """Fastest and less CPU intensive loading method."""
        print(filepath)
        command = ['ffmpeg',
                   '-i', filepath,
                   '-f', 's16le',
                   '-acodec', 'pcm_s16le',
                   '-ac', '1']  # channels: 2 for stereo, 1 for mono
        if self.sampling_rate != SAMPLING_RATE:
            command.extend(['-ar', str(self.sampling_rate)])
        command.append('-')
        # 30s at 44.1 kHz ~= 1.3e6
        proc = sp.run(command, stdout=sp.PIPE, bufsize=10**7, stderr=sp.DEVNULL, check=True)

        return np.fromstring(proc.stdout, dtype="int16")

In [69]:
def build_sample_loader(audio_dir, Y, loader):

    class SampleLoader:

        def __init__(self, tids, batch_size=4):
            self.lock1 = multiprocessing.Lock()
            self.lock2 = multiprocessing.Lock()
            self.batch_foremost = sharedctypes.RawValue(ctypes.c_int, 0)
            self.batch_rearmost = sharedctypes.RawValue(ctypes.c_int, -1)
            self.condition = multiprocessing.Condition(lock=self.lock2)

            data = sharedctypes.RawArray(ctypes.c_int, tids.data)
            self.tids = np.ctypeslib.as_array(data)

            self.batch_size = batch_size
            self.loader = loader
            self.X = np.empty((self.batch_size, *loader.shape))
            self.Y = np.empty((self.batch_size, Y.shape[1]), dtype=np.int)

        def __iter__(self):
            return self

        def __next__(self):

            with self.lock1:
                if self.batch_foremost.value == 0:
                    np.random.shuffle(self.tids)

                batch_current = self.batch_foremost.value
                if self.batch_foremost.value + self.batch_size < self.tids.size:
                    batch_size = self.batch_size
                    self.batch_foremost.value += self.batch_size
                else:
                    batch_size = self.tids.size - self.batch_foremost.value
                    self.batch_foremost.value = 0

#                 print(self.tids, self.batch_foremost.value, batch_current, self.tids[batch_current], batch_size)
                print('queue', self.tids[batch_current], batch_size)
                tids = np.array(self.tids[batch_current:batch_current+batch_size])

            for i, tid in enumerate(tids):
                try:
                    tmp = self.loader.load(get_audio_path(audio_dir, tid))
                    self.X[i] = tmp
                    self.Y[i] = Y.loc[tid]
                except:
                    continue

            with self.lock2:
                while (batch_current - self.batch_rearmost.value) % self.tids.size > self.batch_size:
                    # print('wait', indices[0], batch_current, self.batch_rearmost.value)
                    self.condition.wait()
                self.condition.notify_all()
                # print('yield', indices[0], batch_current, self.batch_rearmost.value)
                self.batch_rearmost.value = batch_current

                return self.X[:batch_size], self.Y[:batch_size]

    return SampleLoader


In [66]:

labels_onehot = LabelBinarizer().fit_transform(tracks['track', 'genre_top'])
labels_onehot = pd.DataFrame(labels_onehot, index=tracks.index)
len(tracks)
labels_onehot

,0,1,2,3,4,5,6,7
track_id,,,,,,,,
2,0,0,0,1,0,0,0,0
5,0,0,0,1,0,0,0,0
10,0,0,0,0,0,0,1,0
140,0,0,1,0,0,0,0,0
141,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...
154308,0,0,0,1,0,0,0,0
154309,0,0,0,1,0,0,0,0
154413,0,0,0,0,0,0,1,0


In [75]:
import warnings
warnings.filterwarnings('ignore')
AUDIO_DIR='/home/hany/MVI/test/cropped_5s'
print('00000000000')
loader = FfmpegLoader(sampling_rate=100)
SampleLoader = build_sample_loader(AUDIO_DIR, labels_onehot, loader)
print('Dimensionality: {}'.format(loader.shape))

print(loader.shape[0])

print(loader)

keras.backend.clear_session()

print('111111111111111111111111')

model = keras.models.Sequential()
model.add(Dense(output_dim=1000, input_shape=loader.shape))
model.add(Activation("relu"))
model.add(Dense(output_dim=100, input_shape=loader.shape))
model.add(Activation("relu"))
model.add(Dense(output_dim=labels_onehot.shape[1]))
model.add(Activation("softmax"))
print('222222222222222222222222')

optimizer = tensorflow.keras.optimizers.SGD(lr=0.1, momentum=0.9, nesterov=True)
model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
print('333333333333333333333333')

print(train.size)
model.fit_generator(SampleLoader(train, batch_size=2), train.size, nb_epoch=2, **params)
loss = model.evaluate_generator(SampleLoader(val, batch_size=2), val.size, **params)
loss = model.evaluate_generator(SampleLoader(test, batch_size=2), test.size, **params)
Y = model.predict_generator(SampleLoader(test, batch_size=2), test.size, **params);

loss

00000000000
Dimensionality: (18,)
18
111111111111111111111111
222222222222222222222222
333333333333333333333333
6397
Epoch 1/2
queue 145742 2
/home/hany/MVI/test/cropped_5s/145742.mp3
queue 49846 2
/home/hany/MVI/test/cropped_5s/049846.mp3
queue 1278 2
/home/hany/MVI/test/cropped_5s/001278.mp3
queue 120330 2
/home/hany/MVI/test/cropped_5s/120330.mp3
queue 47073 2
/home/hany/MVI/test/cropped_5s/047073.mp3
queue 97590 2
/home/hany/MVI/test/cropped_5s/097590.mp3
queue 109684 2
/home/hany/MVI/test/cropped_5s/109684.mp3
queue 107851 2
/home/hany/MVI/test/cropped_5s/107851.mp3
/home/hany/MVI/test/cropped_5s/064093.mp3
/home/hany/MVI/test/cropped_5s/138070.mp3
/home/hany/MVI/test/cropped_5s/051992.mp3
/home/hany/MVI/test/cropped_5s/086443.mp3
/home/hany/MVI/test/cropped_5s/029750.mp3
/home/hany/MVI/test/cropped_5s/003720.mp3
/home/hany/MVI/test/cropped_5s/012376.mp3
/home/hany/MVI/test/cropped_5s/049071.mp3
queue 71937 2
/home/hany/MVI/test/cropped_5s/071937.mp3
queue 124470 2
queue 127279 2


  56/6397 [..............................] - ETA: 6:15 - loss: nan - accuracy: 0.0804/home/hany/MVI/test/cropped_5s/139663.mp3
queue 21677 2
  57/6397 [..............................] - ETA: 6:17 - loss: nan - accuracy: 0.0789/home/hany/MVI/test/cropped_5s/021677.mp3
queue 17636 2
queue 144741 2
/home/hany/MVI/test/cropped_5s/122498.mp3
/home/hany/MVI/test/cropped_5s/017636.mp3
  58/6397 [..............................] - ETA: 6:16 - loss: nan - accuracy: 0.0776/home/hany/MVI/test/cropped_5s/144741.mp3
queue 59671 2
/home/hany/MVI/test/cropped_5s/059671.mp3
/home/hany/MVI/test/cropped_5s/091791.mp3
  61/6397 [..............................] - ETA: 6:04 - loss: nan - accuracy: 0.0902/home/hany/MVI/test/cropped_5s/056639.mp3
queue 93363 2
/home/hany/MVI/test/cropped_5s/093363.mp3
/home/hany/MVI/test/cropped_5s/023015.mp3
/home/hany/MVI/test/cropped_5s/046733.mp3
queue 70654 2
/home/hany/MVI/test/cropped_5s/034258.mp3
  62/6397 [..............................] - ETA: 6:10 - loss: nan - ac

/home/hany/MVI/test/cropped_5s/127722.mp3
queue 52633 2
/home/hany/MVI/test/cropped_5s/052633.mp3
/home/hany/MVI/test/cropped_5s/109355.mp3
/home/hany/MVI/test/cropped_5s/036304.mp3
 117/6397 [..............................] - ETA: 5:20 - loss: nan - accuracy: 0.1282queue 123438 2
queue 145464 2
/home/hany/MVI/test/cropped_5s/123438.mp3
/home/hany/MVI/test/cropped_5s/145464.mp3
/home/hany/MVI/test/cropped_5s/015880.mp3
/home/hany/MVI/test/cropped_5s/120297.mp3
queue 108961 2
 119/6397 [..............................] - ETA: 5:19 - loss: nan - accuracy: 0.1261/home/hany/MVI/test/cropped_5s/108961.mp3
/home/hany/MVI/test/cropped_5s/096401.mp3
 120/6397 [..............................] - ETA: 5:19 - loss: nan - accuracy: 0.1292queue 137173 2
/home/hany/MVI/test/cropped_5s/137173.mp3
queue 122959 2
/home/hany/MVI/test/cropped_5s/122959.mp3
/home/hany/MVI/test/cropped_5s/126221.mp3
/home/hany/MVI/test/cropped_5s/039291.mp3
 122/6397 [..............................] - ETA: 5:17 - loss: nan -

 174/6397 [..............................] - ETA: 4:59 - loss: nan - accuracy: 0.1322queue 44781 2
/home/hany/MVI/test/cropped_5s/044781.mp3
queue 89456 2
 175/6397 [..............................] - ETA: 4:59 - loss: nan - accuracy: 0.1314/home/hany/MVI/test/cropped_5s/089456.mp3
queue 133803 2
/home/hany/MVI/test/cropped_5s/133803.mp3
queue 147265 2
/home/hany/MVI/test/cropped_5s/147265.mp3
queue 98617 2
 179/6397 [..............................] - ETA: 4:54 - loss: nan - accuracy: 0.1285queue 26035 2
/home/hany/MVI/test/cropped_5s/098626.mp3
/home/hany/MVI/test/cropped_5s/098617.mp3
/home/hany/MVI/test/cropped_5s/026035.mp3
 180/6397 [..............................] - ETA: 4:55 - loss: nan - accuracy: 0.1278queue 136091 2
/home/hany/MVI/test/cropped_5s/136091.mp3
/home/hany/MVI/test/cropped_5s/049068.mp3
/home/hany/MVI/test/cropped_5s/007490.mp3
/home/hany/MVI/test/cropped_5s/133967.mp3
 181/6397 [..............................] - ETA: 4:56 - loss: nan - accuracy: 0.1271queue 22481 

/home/hany/MVI/test/cropped_5s/004017.mp3
queue 122499 2
 236/6397 [>.............................] - ETA: 4:43 - loss: nan - accuracy: 0.1123/home/hany/MVI/test/cropped_5s/122499.mp3
queue 144939 2
/home/hany/MVI/test/cropped_5s/111230.mp3
/home/hany/MVI/test/cropped_5s/144939.mp3
/home/hany/MVI/test/cropped_5s/004236.mp3
/home/hany/MVI/test/cropped_5s/109350.mp3
/home/hany/MVI/test/cropped_5s/108155.mp3
/home/hany/MVI/test/cropped_5s/129696.mp3
/home/hany/MVI/test/cropped_5s/088877.mp3
/home/hany/MVI/test/cropped_5s/013748.mp3
 238/6397 [>.............................] - ETA: 4:45 - loss: nan - accuracy: 0.1113queue 45392 2
/home/hany/MVI/test/cropped_5s/045392.mp3
/home/hany/MVI/test/cropped_5s/092885.mp3
queue 142530 2
/home/hany/MVI/test/cropped_5s/142530.mp3
 240/6397 [>.............................] - ETA: 4:44 - loss: nan - accuracy: 0.1104queue 87430 2
/home/hany/MVI/test/cropped_5s/087430.mp3
queue 57360 2
 241/6397 [>.............................] - ETA: 4:44 - loss: nan - a

...

/home/hany/MVI/test/cropped_5s/012059.mp3
/home/hany/MVI/test/cropped_5s/137605.mp3
/home/hany/MVI/test/cropped_5s/039317.mp3
/home/hany/MVI/test/cropped_5s/148612.mp3
/home/hany/MVI/test/cropped_5s/127183.mp3
queue 10442 2
/home/hany/MVI/test/cropped_5s/010442.mp3
/home/hany/MVI/test/cropped_5s/142359.mp3
/home/hany/MVI/test/cropped_5s/074393.mp3
queue 142537 2
/home/hany/MVI/test/cropped_5s/142537.mp3
queue 110085 2
/home/hany/MVI/test/cropped_5s/110085.mp3
queue 103522 2
/home/hany/MVI/test/cropped_5s/103522.mp3
queue 40243 2
queue 127189 2
/home/hany/MVI/test/cropped_5s/040243.mp3
/home/hany/MVI/test/cropped_5s/127189.mp3
/home/hany/MVI/test/cropped_5s/067333.mp3
queue 28072 2
/home/hany/MVI/test/cropped_5s/028072.mp3
queue 115294 2
/home/hany/MVI/test/cropped_5s/147021.mp3
/home/hany/MVI/test/cropped_5s/115294.mp3
/home/hany/MVI/test/cropped_5s/012488.mp3
/home/hany/MVI/test/cropped_5s/112487.mp3
/home/hany/MVI/test/cropped_5s/016338.mp3
/home/hany/MVI/test/cropped_5s/046720.mp3
q

queue 39605 2
queue 57274 2
/home/hany/MVI/test/cropped_5s/039605.mp3
/home/hany/MVI/test/cropped_5s/057274.mp3
queue 69221 2
/home/hany/MVI/test/cropped_5s/069221.mp3
queue 12691 2
queue 106872 2
/home/hany/MVI/test/cropped_5s/012691.mp3
queue 108528 2
/home/hany/MVI/test/cropped_5s/106872.mp3
/home/hany/MVI/test/cropped_5s/108528.mp3
/home/hany/MVI/test/cropped_5s/068536.mp3
queue 110768 2
/home/hany/MVI/test/cropped_5s/110768.mp3
/home/hany/MVI/test/cropped_5s/092366.mp3
/home/hany/MVI/test/cropped_5s/111151.mp3
/home/hany/MVI/test/cropped_5s/145762.mp3
/home/hany/MVI/test/cropped_5s/045934.mp3
/home/hany/MVI/test/cropped_5s/127559.mp3
/home/hany/MVI/test/cropped_5s/066535.mp3
queue 12058 2
/home/hany/MVI/test/cropped_5s/012058.mp3
queue 60537 2
/home/hany/MVI/test/cropped_5s/060537.mp3
queue 144592 2
/home/hany/MVI/test/cropped_5s/076130.mp3
/home/hany/MVI/test/cropped_5s/144592.mp3
queue 7377 2
/home/hany/MVI/test/cropped_5s/007377.mp3
queue 150288 2
/home/hany/MVI/test/cropped_5s

queue 32435 2
/home/hany/MVI/test/cropped_5s/032435.mp3
/home/hany/MVI/test/cropped_5s/011019.mp3
queue 90529 2
/home/hany/MVI/test/cropped_5s/090529.mp3
/home/hany/MVI/test/cropped_5s/145458.mp3
/home/hany/MVI/test/cropped_5s/040233.mp3
/home/hany/MVI/test/cropped_5s/137624.mp3
queue 112483 2
/home/hany/MVI/test/cropped_5s/112483.mp3
queue 97286 2
queue 10679 2
/home/hany/MVI/test/cropped_5s/042374.mp3
/home/hany/MVI/test/cropped_5s/010679.mp3
/home/hany/MVI/test/cropped_5s/097286.mp3
/home/hany/MVI/test/cropped_5s/144215.mp3
queue 12508 2
/home/hany/MVI/test/cropped_5s/012508.mp3
queue 113273 2
/home/hany/MVI/test/cropped_5s/154309.mp3
queue 119828 2
/home/hany/MVI/test/cropped_5s/119828.mp3
/home/hany/MVI/test/cropped_5s/113273.mp3
queue 16340 2
/home/hany/MVI/test/cropped_5s/016340.mp3
/home/hany/MVI/test/cropped_5s/073776.mp3
/home/hany/MVI/test/cropped_5s/010678.mp3
queue 85594 2
/home/hany/MVI/test/cropped_5s/085594.mp3
/home/hany/MVI/test/cropped_5s/128886.mp3
/home/hany/MVI/te

[nan, 0.125]

queue 16995 2
/home/hany/MVI/test/cropped_5s/140625.mp3
/home/hany/MVI/test/cropped_5s/016995.mp3
queue 35204 2
/home/hany/MVI/test/cropped_5s/035204.mp3
/home/hany/MVI/test/cropped_5s/040844.mp3
queue 63065 2
/home/hany/MVI/test/cropped_5s/063065.mp3
/home/hany/MVI/test/cropped_5s/121913.mp3
/home/hany/MVI/test/cropped_5s/013328.mp3
/home/hany/MVI/test/cropped_5s/110743.mp3
/home/hany/MVI/test/cropped_5s/154303.mp3
/home/hany/MVI/test/cropped_5s/068582.mp3


In [74]:
Y

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)

### 2.2 Convolutional neural network

* Architecture: [End-to-end learning for music audio](http://www.mirlab.org/conference_papers/International_Conference/ICASSP%202014/papers/p7014-dieleman.pdf), Sander Dieleman, Benjamin Schrauwen.
* Missing: track segmentation and class averaging (majority voting)
* Compared with log-scaled mel-spectrograms instead of strided convolution as first layer.
* Larger net: http://benanne.github.io/2014/08/05/spotify-cnns.html

In [ ]:
loader = utils.LibrosaLoader(sampling_rate=16000)
#loader = utils.LibrosaLoader(sampling_rate=16000)
SampleLoader = utils.build_sample_loader(AUDIO_DIR, labels_onehot, loader)

keras.backend.clear_session()

model = keras.models.Sequential()
model.add(Reshape((-1, 1), input_shape=loader.shape))
print(model.output_shape)

model.add(Conv1D(128, 512, subsample_length=512))
print(model.output_shape)
model.add(Activation("relu"))

model.add(Conv1D(32, 8))
print(model.output_shape)
model.add(Activation("relu"))
model.add(MaxPooling1D(4))

model.add(Conv1D(32, 8))
print(model.output_shape)
model.add(Activation("relu"))
model.add(MaxPooling1D(4))

print(model.output_shape)
model.add(Dropout(0.25))
model.add(Flatten())
print(model.output_shape)
# model.add(Dense(100))
model.add(Activation("relu"))
print(model.output_shape)
model.add(Dense(labels_onehot.shape[1]))
model.add(Activation("softmax"))
print(model.output_shape)

optimizer = keras.optimizers.SGD(lr=0.01, momentum=0.9, nesterov=True)
#optimizer = keras.optimizers.Adam()#lr=1e-5)#, momentum=0.9, nesterov=True)
model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.fit_generator(SampleLoader(train, batch_size=10), train.size, nb_epoch=20, **params)
loss = model.evaluate_generator(SampleLoader(val, batch_size=10), val.size, **params)
loss = model.evaluate_generator(SampleLoader(test, batch_size=10), test.size, **params)

loss

### 2.3 Recurrent neural network

## 3 Deep learning on extracted audio features

Look at:
* Pre-processing in Keras: https://github.com/keunwoochoi/kapre
* Convolutional Recurrent Neural Networks for Music Classification: https://github.com/keunwoochoi/icassp_2017
* Music Auto-Tagger: https://github.com/keunwoochoi/music-auto_tagging-keras
* Pre-processor: https://github.com/bmcfee/pumpp

### 3.1 ConvNet on MFCC

* Architecture: [Automatic Musical Pattern Feature Extraction Using Convolutional Neural Network](http://www.iaeng.org/publication/IMECS2010/IMECS2010_pp546-550.pdf), Tom LH. Li, Antoni B. Chan and Andy HW. Chun
* Missing: track segmentation and majority voting.
* Best seen: 17.6%

In [ ]:
class MfccLoader(utils.Loader):
    raw_loader = utils.FfmpegLoader(sampling_rate=22050)
    #shape = (13, 190)  # For segmented tracks.
    shape = (13, 2582)
    def load(self, filename):
        import librosa
        x = self.raw_loader.load(filename)
        # Each MFCC frame spans 23ms on the audio signal with 50% overlap with the adjacent frames.
        mfcc = librosa.feature.mfcc(x, sr=22050, n_mfcc=13, n_fft=512, hop_length=256)
        return mfcc

loader = MfccLoader()
SampleLoader = utils.build_sample_loader(AUDIO_DIR, labels_onehot, loader)
print(utils.get_audio_path(AUDIO_DIR, 2))
loader.load(utils.get_audio_path(AUDIO_DIR, 2))[0].shape

In [ ]:
keras.backend.clear_session()

model = keras.models.Sequential()
model.add(Reshape((*loader.shape, 1),  input_shape=loader.shape))
print(model.output_shape)

model.add(Conv2D(3, 13, 10, subsample=(1, 4)))
model.add(Activation("relu"))
print(model.output_shape)

model.add(Conv2D(15, 1, 10, subsample=(1, 4)))
model.add(Activation("relu"))
print(model.output_shape)

model.add(Conv2D(65, 1, 10, subsample=(1, 4)))
model.add(Activation("relu"))
print(model.output_shape)

model.add(Flatten())
print(model.output_shape)
model.add(Dense(labels_onehot.shape[1]))
model.add(Activation("softmax"))
print(model.output_shape)

optimizer = keras.optimizers.SGD(1e-3)#lr=0.01, momentum=0.9, nesterov=True)
#optimizer = keras.optimizers.Adam()#lr=1e-5)#
model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.fit_generator(SampleLoader(train, batch_size=16), train.size, nb_epoch=20, **params)
loss = model.evaluate_generator(SampleLoader(val, batch_size=16), val.size, **params)
loss = model.evaluate_generator(SampleLoader(test, batch_size=16), test.size, **params)
#Y = model.predict_generator(loader, test.size, pickle_safe=True, nb_worker=NB_WORKER, max_q_size=5)

loss